In [1]:
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [2]:
df = pd.read_csv('0507 df_for_holdout.csv')

In [3]:
df.set_index('Date',inplace = True)

In [4]:
df.head()

,시언감성점수,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,시언서술포함감성점수,주택,아파트,분양,서울,가구,부동산,단지,시장,next_spi
Date,,,,,,,,,,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,-1.635618e+08,1160,1094,666,729,1180,685,461,459,82.5
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,-1.777621e+08,1160,1101,639,652,1265,669,383,496,83.4
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,-1.980306e+08,1341,1102,1014,607,961,659,576,578,83.9
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,-1.898849e+08,1405,1186,977,658,1031,720,534,577,84.1
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,-2.402702e+08,1537,1543,1267,854,1245,1013,770,756,84.4


In [10]:
min(df.next_spi[90:])

99.5

# 전처리 및 Train/Test Set 분할

In [ ]:
X = df.iloc[:,:-1]
y = df['next_spi'].to_list()

In [ ]:
X.head()

In [ ]:
# Holdout으로 Train/Test 분할
X_train_temp = X.iloc[:90]
y_train_temp = y[:90]
X_test = X.iloc[90:]
y_test = y[90:]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_temp, y_train_temp, test_size = 0.25, random_state = 0)

tf_idf_words = ['주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장']

# 거시경제변수만 포함된 Case

In [ ]:
# 거시경제변수만 포함된 X 만들기
X_train_basic = X_train.drop(['시언감성점수','효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'], axis = 1)
X_valid_basic = X_valid.drop(['시언감성점수','효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'], axis = 1)
X_test_basic = X_test.drop(['시언감성점수','효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'], axis = 1)

In [ ]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {
        'n_estimators' : [50, 100, 150, 200],
        'gamma': [0.3, 0.6, 0.9, 1.2, 1.5],
        'subsample' : [i/10.0 for i in range(6,11)],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9]
        }
  
grid_XGBR = GridSearchCV(XGBRegressor(), param_grid, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_XGBR.fit(X_train_basic, y_train)

In [ ]:
# valid에 대한 성능
y_pred = grid_XGBR.predict(X_valid_basic)
RMSE = mean_squared_error(y_valid, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

In [ ]:
# Test에 대한 성능
y_pred = grid_XGBR.predict(X_test_basic)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

# 시언 감성 점수 Case

## 시언 감성 점수 데이터 전처리

In [ ]:
# 시언 감성 점수 (명사만 이용) 포함 X 만들기
X_train_si = X_train.drop(['효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_valid_si = X_valid.drop(['효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_si = X_test.drop(['효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

In [ ]:
X_train_si_verb = X_train.drop(['시언감성점수','효준감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_valid_si_verb = X_valid.drop(['시언감성점수','효준감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_si_verb = X_test.drop(['시언감성점수','효준감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

## (시언 감성 점수 (명사)) 이용 SVR 수립 및 평가

In [ ]:
# defining parameter range
param_grid = {
        'n_estimators' : [50, 100, 150, 200],
        'gamma': [0.3, 0.6, 0.9, 1.2, 1.5],
        'subsample' : [i/10.0 for i in range(6,11)],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9]
        }
  
grid_XGBR = GridSearchCV(XGBRegressor(), param_grid, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_XGBR.fit(X_train_si, y_train)

In [ ]:
# valid에 대한 성능
y_pred = grid_XGBR.predict(X_valid_si)
RMSE = mean_squared_error(y_valid, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

In [ ]:
# test에 대한 성능
y_pred = grid_XGBR.predict(X_test_si)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

## (시언 감성 점수 (서술어 포함)) 이용 SVR 수립 및 평가

In [ ]:
# defining parameter range
param_grid = {
        'n_estimators' : [50, 100, 150, 200],
        'gamma': [0.3, 0.6, 0.9, 1.2, 1.5],
        'subsample' : [i/10.0 for i in range(6,11)],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9]
        }
  
grid_XGBR = GridSearchCV(XGBRegressor(), param_grid, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_XGBR.fit(X_train_si_verb, y_train)

In [ ]:
# valid에 대한 성능 
y_pred = grid_XGBR.predict(X_valid_si_verb)
RMSE = mean_squared_error(y_valid, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

In [ ]:
# test에 대한 성능 
y_pred = grid_XGBR.predict(X_test_si_verb)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

# 효준 감성 점수(명사) Case

## 효준 감성 점수 포함 성능

In [ ]:
X_train_hj = X_train.drop(['시언감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_valid_hj = X_valid.drop(['시언감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_hj = X_test.drop(['시언감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

In [ ]:
# defining parameter range
param_grid = {
        'n_estimators' : [50, 100, 150, 200],
        'gamma': [0.3, 0.6, 0.9, 1.2, 1.5],
        'subsample' : [i/10.0 for i in range(6,11)],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9]
        }
  
grid_XGBR = GridSearchCV(XGBRegressor(), param_grid, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_XGBR.fit(X_train_hj, y_train)

In [ ]:
# valid에 대한 성능 
y_pred = grid_XGBR.predict(X_valid_hj)
RMSE = mean_squared_error(y_valid, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

In [ ]:
# test에 대한 성능 
y_pred = grid_XGBR.predict(X_test_hj)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

# TF-IDF 상위 8개 단어 빈도수 추가 Case

In [ ]:
X_train_tf_idf = X_train.drop(columns = [], axis = 1)
X_valid_tf_idf = X_valid.drop(columns = [], axis = 1)
X_test_tf_idf = X_test.drop(columns = [], axis = 1)

In [ ]:
# defining parameter range
param_grid = {
        'n_estimators' : [50, 100, 150, 200],
        'gamma': [0.3, 0.6, 0.9, 1.2, 1.5],
        'subsample' : [i/10.0 for i in range(6,11)],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9]
        }
  
grid_XGBR = GridSearchCV(XGBRegressor(), param_grid, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_XGBR.fit(X_train_tf_idf, y_train)

In [ ]:
# valid에 대한 성능
y_pred = grid_XGBR.predict(X_valid_tf_idf)
RMSE = mean_squared_error(y_valid, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred

In [ ]:
# test에 대한 성능
y_pred = grid_XGBR.predict(X_test_tf_idf)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_XGBR.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_XGBR.best_estimator_)

In [ ]:
del param_grid, grid_XGBR, RMSE, y_pred